<div style="position: relative; text-align: center; padding: 30px;">
  <h1><strong>Gestión en Logística y Cadena de Suministro</strong></h1>
  <h3><strong>Ejercicio 4</strong></h3>
</div>

El sudoku es un juego japonés. El objetivo del juego es rellenar con los números naturales del 1 al 9 cada una de las celdas vacías de una cuadrícula de $9 \times 9$, dividida en 9 sub-cuadrículas de $3 \times 3$, y que ha sido parcialmente rellenada con algunos valores en algunas de las celdas. El objetivo del juego es rellenar las celdas vacías de manera tal que en cada una de las filas, en cada una de las columnas y en cada una de las submatrices no debe haber números repetidos. Se requiere formular el problema de resolver un sudoku como un problema de programación matemática.
